<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Building-level-analysis" data-toc-modified-id="Building-level-analysis-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Building-level analysis</a></span></li><li><span><a href="#Total-load-breakdown-by-building-type" data-toc-modified-id="Total-load-breakdown-by-building-type-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Total load breakdown by building type</a></span></li><li><span><a href="#Forecasting" data-toc-modified-id="Forecasting-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Forecasting</a></span></li></ul></div>

In [ ]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
print("Executable you are using")
print(sys.executable)
from sesibuildings import config
print("Your DATA_PATH:")
print(config["DATA_PATH"])

Executable you are using
/Users/lucasfuentes/anaconda3/bin/python
Your DATA_PATH:
/Volumes/GoogleDrive/My Drive/sesi-buildings/data/


In [2]:
import pandas as pd
import numpy as np
import re
import os
from sesibuildings.data.load_clean_data import load_meters,change_columns_names
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from sesibuildings.viz.gui_objects import GUIBaseViz, GUIStackedPlot, GUIPredViz

In [3]:
# Load data
nrows = None  # pass in None here toload all rows

df_c = load_meters(meters="buildings", building_type="C", clean=True, nrows=nrows)

df_c_0=load_meters(meters="buildings", building_type="C", clean=False, nrows = nrows)
df_c_0=df_c_0.diff()

df_c=change_columns_names(df_c)
df_c_0=change_columns_names(df_c_0)

df_T=load_meters(meters="temperature", clean=False)

df_cef = load_meters("cef", clean=True)

/Users/lucasfuentes/Documents/Benson/Code/sesi-buildings/src/sesibuildings/data/load_clean_data.py:181: RuntimeWarning: invalid value encountered in less
  bb = np.where(aa < 0)[0]


# Building-level analysis

In [7]:
gui = GUIBaseViz(df_c, df_c_0)
gui.make_viz()
gui.display()

Output()

# Total load breakdown by building type

In [ ]:
# Recreate aggregated dataset

from sesibuildings.data.meter_ref import building_to_meter
bm = building_to_meter()
# Recreate building profiles
building_profile = dict()
for col in df_c.columns:
    matched = bm[bm["Building Name"] == col]
    if len(matched) == 1:
        building_profile[col] = matched["Building Profile"].values[0]
    elif ("E.D. STONE" in col) or ("NURSING POD" in col) or ("BECKMAN CENTER" in col) or ("hospital" in col.lower()):
            building_profile[col] = "Hospital"
    else:
        building_profile[col] = "NA"
#         print(col)
building_meta = pd.DataFrame.from_dict(building_profile, orient="index", columns=["profile"])
df_agg = pd.DataFrame(index=df_c.index)

for type_ in building_meta.profile.unique():
    columns = building_meta[building_meta.profile == type_].index
    df_agg[type_] = df_c[[col for col in columns]].sum(axis=1)

In [ ]:
gui = GUIStackedPlot(df_agg, df_cef)
gui.make_viz()
gui.display()

# Forecasting

In [4]:
#prepare the data set for Gaussian forecaster for now
df_T=pd.DataFrame(df_T,columns=['LOKEY_OAT'])
df_T['day']=df_T.index.date
df_T=df_T.groupby('day').max()
df_T.index=pd.to_datetime(df_T.index)

In [5]:
gui = GUIPredViz(df_c, df_T)
gui.make_viz()
gui.display()

EOFError: Ran out of input

In [25]:
from sesibuildings.forecast.helpers_testing import load_pretrained

In [26]:
model, training_end_date = load_pretrained('Gaussian')

EOFError: Ran out of input